git checkout master   
git branch main master -f    
git checkout main  
git push origin main -f 


In [3]:
import pandas as pd 
import os 
pd.set_option('display.max_columns', None)  # Tüm sütunları gösterir
pd.set_option('display.max_colwidth', None) # Sütun genişliklerini sınırlandırmadan gösterir

In [25]:
#fbrefscrape ile scrape edilen maç linkleri df inden istediğimiz linkleri ayıklayıp filtered_week_match_links.txt olarak scrape e gönderir.
def fbrefscrape(jsonname):
    df = pd.read_csv('/Users/icy/Football_project/Footbal_project/csvler/last_df_.csv', parse_dates=['date'])
    week_match_links = pd.read_json(f'/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/{jsonname}.json')  
    all_team_names = list(set(list(df.home_team.unique()) + list(df.away_team.unique()))) # -> datada bulunan tum takimlar. linklerden istediklerimizi çekmek için lazım olcak.
    eklenmesi_gereken_takimlar = ['fenerbahce','adana-demirspor','besiktas','eyupspor','go-ahead','willem-ii','paris-saint-germain','le-havre','futebol-sad','gil-vicente','west-ham-united',
                                  'aston-villa','saint-etienne','as-monaco','nec-nijmegen','az-alkmaar','real-sociedad','rayo-vallecano','estrela-amadora','famalicao','real-valladolid','real-madrid',
                                  'las-palmas','leganes','ipswich-town','manchester-city','mainz-05','union-berlin','casa-pia','santa-clara']   
    all_team_names = all_team_names + eklenmesi_gereken_takimlar 
    all_team_names.remove('NEC')
    all_team_names.remove('AZ')
    all_team_names.remove('ADO')
    all_team_names.remove('PSG')
    filtered_week_match_links = week_match_links[week_match_links['url'].str.contains('|'.join(all_team_names), case=False)]
    filtered_week_match_links = list(filtered_week_match_links.url.values)
    # for eleman in filtered_week_match_links : 
    #     if 'rangersu21' in eleman or 'bo-rangers' in eleman or 'enugu-rangers' in eleman or 'cucuta-deportivo' in eleman or 'internacional' in eleman or 'deportivo-mongomo' in eleman or \
    #         'club-nacional-de' in eleman or 'cska-1948' in eleman or 'bo-rangers' in eleman:
    #         filtered_week_match_links.remove(eleman)
    # # Kelimelerin bir set içinde tanımlanması
    keywords = {'rangersu21', 'bo-rangers', 'enugu-rangers', 'cucuta-deportivo', 'internacional', 'deportivo-mongomo', 'club-nacional-de', 'cska-1948'}
    # Filtreleme işlemi
    filtered_week_match_links = [eleman for eleman in filtered_week_match_links if not any(keyword in eleman for keyword in keywords)]


    directory = '/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/sofa_temiz_urls'
    file_path = os.path.join(directory, f'filtered_week_match_links.txt')
    with open(file_path, 'w') as file:
        for url in filtered_week_match_links:
            file.write(url + '\n')
fbrefscrape(jsonname= 'week_match_links')

In [106]:
# BURAYI BASMADAN ONCE GEREKLI VERILERDE EKSIK VAR MI KONTROL ET DUZELT (EKSIK SARI VE KIRMIZI KART SORUN DEGIL)
import pandas as pd 
import os 
df = pd.read_csv('/Users/icy/Desktop/sofa/sofa_all_works/csvler/last_df.csv', parse_dates=['date'])
week_matches = pd.read_json('/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/week_matches_first.json',convert_dates= False)
week_matches_first_half = pd.read_json('/Users/icy/Desktop/scrapping_scrapy_football/football/football/spiders/week_matches_second.json')
merged_week_matches = pd.merge(week_matches,week_matches_first_half, on= 'match_link', how='left')
merged_week_matches = merged_week_matches[merged_week_matches.Organizasyon.isin(df.organizasyon.unique())] #-> istediğim organizsayon isimlerni çeker.
merged_week_matches = merged_week_matches[merged_week_matches.ulke.isin(df.ulke.unique())] #-> ulkeler icin tekrar bi ayikladim bazen bir organizasyon iki ulkede olabiliyor.
# kart verileri eksikse 0 oldugundan eksik oluyorlar onları 0 yaptım 
merged_week_matches['Yellow cards_home'] = merged_week_matches['Yellow cards_home'].fillna(0).astype(int)
merged_week_matches['Yellow cards_away'] = merged_week_matches['Yellow cards_away'].fillna(0).astype(int)
merged_week_matches['Red cards_home'] = merged_week_matches['Red cards_home'].fillna(0).astype(int)
merged_week_matches['Red cards_away'] = merged_week_matches['Red cards_away'].fillna(0).astype(int)
merged_week_matches['Yellow cards_home_firsthalf'] = merged_week_matches['Yellow cards_home_firsthalf'].fillna(0).astype(int)
merged_week_matches['Yellow cards_away_firsthalf'] = merged_week_matches['Yellow cards_away_firsthalf'].fillna(0).astype(int)
merged_week_matches['Red cards_home_firsthalf'] = merged_week_matches['Red cards_home_firsthalf'].fillna(0).astype(int)
merged_week_matches['Red cards_away_firsthalf'] = merged_week_matches['Red cards_away_firsthalf'].fillna(0).astype(int)
merged_week_matches['date'] = pd.to_datetime(merged_week_matches['date'], format='%d-%m-%Y') #--> date todatetime a cevrildi
merged_week_matches.columns = merged_week_matches.columns.str.lower().str.replace(' ', '_') #--> sutun isimleri kucuk harf oldu ve bosluklar alt tire ile degisti.
# season_round ve season_period sutunlarını olusturur.
df_leagues = merged_week_matches[~(merged_week_matches.ulke == 'Europe')]
df_leagues.loc[ : , 'season_round'] = df_leagues.org_detay.apply(lambda x: x[-2:]).astype(int)
df_leagues = df_leagues[['match_link','season_round']]
df_leagues.loc[df_leagues.season_round <= 9 , 'season_period'] = 1
df_leagues.loc[(df_leagues.season_round > 9) & (df_leagues.season_round <= 18)  , 'season_period'] = 2
df_leagues.loc[(df_leagues.season_round > 18) & (df_leagues.season_round <= 27)  , 'season_period'] = 3
df_leagues.loc[(df_leagues.season_round >= 28)  , 'season_period'] = 4
merged_week_matches = pd.merge(merged_week_matches,df_leagues, on= 'match_link', how= 'left')
merged_week_matches = merged_week_matches.drop_duplicates()
merged_week_matches = merged_week_matches.fillna(0)
merged_week_matches.season_period = merged_week_matches.season_period.astype(int)
merged_week_matches.season_round = merged_week_matches.season_round.astype(int)
merged_week_matches.reset_index(drop= True, inplace= True)
# ikincyi yari sutunları olusturur.
merged_week_matches['total_shots_away_secondhalf'] = merged_week_matches['total_shots_away'] - merged_week_matches['total_shots_away_firsthalf']
merged_week_matches['total_shots_home_secondhalf'] = merged_week_matches['total_shots_home'] - merged_week_matches['total_shots_home_firsthalf']
merged_week_matches['shots_on_target_home_secondhalf'] = merged_week_matches['shots_on_target_home'] - merged_week_matches['shots_on_target_home_firsthalf']
merged_week_matches['shots_on_target_away_secondhalf'] = merged_week_matches['shots_on_target_away'] - merged_week_matches['shots_on_target_away_firsthalf']
merged_week_matches['shots_inside_box_home_secondhalf'] = merged_week_matches['shots_inside_box_home'] - merged_week_matches['shots_inside_box_home_firsthalf']
merged_week_matches['shots_inside_box_away_secondhalf'] = merged_week_matches['shots_inside_box_away'] - merged_week_matches['shots_inside_box_away_firsthalf']
merged_week_matches['shots_outside_box_home_secondhalf'] = merged_week_matches['shots_outside_box_home'] - merged_week_matches['shots_outside_box_home_firsthalf']
merged_week_matches['shots_outside_box_away_secondhalf'] = merged_week_matches['shots_outside_box_away'] - merged_week_matches['shots_outside_box_away_firsthalf']
merged_week_matches['corner_kicks_home_secondhalf'] = merged_week_matches['corner_kicks_home'] - merged_week_matches['corner_kicks_home_firsthalf']
merged_week_matches['corner_kicks_away_secondhalf'] = merged_week_matches['corner_kicks_away'] - merged_week_matches['corner_kicks_away_firsthalf']
merged_week_matches['passes_home_secondhalf'] = merged_week_matches['passes_home'] - merged_week_matches['passes_home_firsthalf']
merged_week_matches['passes_away_secondhalf'] = merged_week_matches['passes_away'] - merged_week_matches['passes_away_firsthalf']
merged_week_matches['yellow_cards_home_secondhalf'] = merged_week_matches['yellow_cards_home'] - merged_week_matches['yellow_cards_home_firsthalf']
merged_week_matches['yellow_cards_away_secondhalf'] = merged_week_matches['yellow_cards_away'] - merged_week_matches['yellow_cards_away_firsthalf']
merged_week_matches['red_cards_home_secondhalf'] = merged_week_matches['red_cards_home'] - merged_week_matches['red_cards_home_firsthalf']
merged_week_matches['red_cards_away_secondhalf'] = merged_week_matches['red_cards_away'] - merged_week_matches['red_cards_away_firsthalf']
merged_week_matches['free_kicks_home_secondhalf'] = merged_week_matches['free_kicks_home'] - merged_week_matches['free_kicks_home_firsthalf']
merged_week_matches['free_kicks_away_secondhalf'] = merged_week_matches['free_kicks_away'] - merged_week_matches['free_kicks_away_firsthalf']
merged_week_matches[['home_goal','away_goal','home_iy_goal','away_iy_goal']] = merged_week_matches[['home_goal','away_goal','home_iy_goal','away_iy_goal']].astype(int)
merged_week_matches.loc[(merged_week_matches.home_goal> 0) & (merged_week_matches.away_goal> 0),'kg_var'] = 1
merged_week_matches['kg_var'].fillna(0, inplace= True)
merged_week_matches['total_shots'] = merged_week_matches.total_shots_away + merged_week_matches.total_shots_home
merged_week_matches['total_shots_firsthalf'] = merged_week_matches.total_shots_home_firsthalf + merged_week_matches.total_shots_away_firsthalf
merged_week_matches['total_shots_secondhalf'] = merged_week_matches.total_shots - merged_week_matches.total_shots_firsthalf
merged_week_matches['corner_kick_total'] = merged_week_matches.corner_kicks_home + merged_week_matches.corner_kicks_away
merged_week_matches['corner_kick_total_firsthalf'] = merged_week_matches.corner_kicks_home_firsthalf + merged_week_matches.corner_kicks_away_firsthalf
merged_week_matches['corner_kick_total_secondhalf'] = merged_week_matches.corner_kicks_home_secondhalf + merged_week_matches.corner_kicks_away_secondhalf
merged_week_matches['yellow_cards_total'] = merged_week_matches.yellow_cards_home + merged_week_matches.yellow_cards_away
merged_week_matches['red_cards_total'] = merged_week_matches.red_cards_home + merged_week_matches.red_cards_away
merged_week_matches['yellow_cards_firsthalf_total'] = merged_week_matches.yellow_cards_home_firsthalf + merged_week_matches.yellow_cards_away_firsthalf
merged_week_matches['yellow_cards_secondhalf_total'] = merged_week_matches.yellow_cards_home_secondhalf + merged_week_matches.yellow_cards_away_secondhalf
merged_week_matches['red_cards_firsthalf_total'] = merged_week_matches.red_cards_home_firsthalf + merged_week_matches.red_cards_away_firsthalf
merged_week_matches['red_cards_secondhalf_total'] = merged_week_matches.red_cards_home_secondhalf + merged_week_matches.red_cards_away_secondhalf
merged_week_matches.loc[(merged_week_matches.home_goal + merged_week_matches.away_goal) > 1, '1.5o'] = 1 
merged_week_matches.loc[(merged_week_matches.home_goal + merged_week_matches.away_goal) > 2, '2.5o'] = 1 
merged_week_matches['1.5o'].fillna(0, inplace= True)
merged_week_matches['2.5o'].fillna(0, inplace= True)
#--> 2024-2025 sezonu season 8 olarak alıncak
merged_week_matches['season'] = 8 
# result ve iy_result sütunaları
merged_week_matches['result'] = merged_week_matches.apply(lambda row: 1 if row['home_goal'] > row['away_goal'] 
                        else 2 if row['home_goal'] < row['away_goal'] 
                        else 0, axis=1)
merged_week_matches['iy_result'] = merged_week_matches.apply(lambda row: 1 if row['home_iy_goal'] > row['away_iy_goal'] 
                        else 2 if row['home_goal'] < row['away_goal'] 
                        else 0, axis=1)
# istedigim sutunları aldım sadece 
merged_week_matches = merged_week_matches[['date', 'hour', 'ulke', 'organizasyon', 'org_detay', 'home_team','home_goal', 'away_goal', 'away_team', 'result', 'home_iy_goal','away_iy_goal', 
'iy_result', 'corner_kicks_home','corner_kicks_home_firsthalf', 'corner_kicks_away','corner_kicks_away_firsthalf', 'total_shots_home','total_shots_home_firsthalf', 'shots_on_target_home',
'shots_on_target_home_firsthalf', 'total_shots_away','total_shots_away_firsthalf', 'shots_on_target_away','shots_on_target_away_firsthalf', 'fouls_home', 'fouls_away','yellow_cards_home', 
'yellow_cards_home_firsthalf', 'yellow_cards_away','yellow_cards_away_firsthalf', 'red_cards_home','red_cards_home_firsthalf', 'red_cards_away','red_cards_away_firsthalf', 'referee','shots_inside_box_home', 'shots_inside_box_away','shots_outside_box_home', 'shots_outside_box_away', 'free_kicks_home','free_kicks_away', 'ball_possession_home', 'ball_possession_away',
'passes_home', 'passes_away', 'ball_possession_home_firsthalf','ball_possession_away_firsthalf', 'passes_home_firsthalf','passes_away_firsthalf', 'free_kicks_home_firsthalf','free_kicks_away_firsthalf',
'shots_inside_box_home_firsthalf','shots_inside_box_away_firsthalf', 'shots_outside_box_home_firsthalf','shots_outside_box_away_firsthalf', 'total_shots_home_secondhalf','total_shots_away_secondhalf', 
'shots_on_target_home_secondhalf','shots_on_target_away_secondhalf', 'shots_inside_box_home_secondhalf','shots_inside_box_away_secondhalf', 'shots_outside_box_home_secondhalf','corner_kicks_home_secondhalf', 
'corner_kicks_away_secondhalf','passes_home_secondhalf', 'passes_away_secondhalf','free_kicks_home_secondhalf', 'yellow_cards_home_secondhalf','yellow_cards_away_secondhalf', 'red_cards_home_secondhalf',
'red_cards_away_secondhalf', 'free_kicks_away_secondhalf','point_home', 'place_home','point_away','place_away','match_link','season', 'season_round', 'season_period','kg_var','corner_kick_total', 
'corner_kick_total_firsthalf','corner_kick_total_secondhalf', 'total_shots_firsthalf', 'total_shots','total_shots_secondhalf','yellow_cards_total', 'red_cards_total', 
'yellow_cards_firsthalf_total','yellow_cards_secondhalf_total', 'red_cards_firsthalf_total','red_cards_secondhalf_total', '1.5o', '2.5o']]
# dtypeları ayarlarma 
merged_week_matches.hour = merged_week_matches.hour.str.replace(':','.').astype(float).astype(int)
merged_week_matches.ball_possession_home_firsthalf = merged_week_matches.ball_possession_home_firsthalf.str.replace('%','').astype(int)
merged_week_matches.ball_possession_away_firsthalf = merged_week_matches.ball_possession_away_firsthalf.str.replace('%','').astype(int)
merged_week_matches.ball_possession_away = merged_week_matches.ball_possession_away.str.replace('%','').astype(int)
merged_week_matches.ball_possession_home = merged_week_matches.ball_possession_home.str.replace('%','').astype(int)
merged_week_matches[['point_home', 'place_home','point_away','place_away']] = merged_week_matches[['point_home', 'place_home','point_away','place_away']].astype(int)
# concat islemi
df_concanated = pd.concat([df,merged_week_matches])
df_concanated.sort_values(by= 'date', ascending= False, inplace= True)
df_concanated.columns = df_concanated.columns.str.lower()
df_concanated.columns = df_concanated.columns.str.replace(' ','_')
# kolon gerektikce burdan ekle
df_concanated.loc[df_concanated.ulke == 'Europe', 'league_match'] = 0
df_concanated.league_match.fillna(1,inplace= True)
df_concanated.league_match = df_concanated.league_match.astype(int)
#---
df_concanated.loc[:, 'match_round'] = df_concanated['org_detay']
df_concanated.match_round = df_concanated.match_round.str.replace('Round','')
df_concanated.match_round = df_concanated.match_round.str.replace('Final','0')
df_concanated.match_round = df_concanated.match_round.str.replace('Quarterfinal','0')
df_concanated.match_round = df_concanated.match_round.str.replace('Semifinal','0')
df_concanated.match_round = df_concanated.match_round.str.replace(' of 16','0')
df_concanated.match_round = df_concanated.match_round.str.strip()
df_concanated.match_round = df_concanated.match_round.astype(int)
df_concanated.loc[df_concanated['league_match'] == 0 , 'match_round'] = 0
df_concanated = df_concanated[~(df_concanated.duplicated())]
#floatları int yapma
float_columns = df_concanated.select_dtypes(include='float').columns
df_concanated[float_columns] = df_concanated[float_columns].astype(int)

# deneme

# eski df'i yedege alma
df.to_csv('/Users/icy/Desktop/sofa/sofa_all_works/csvler/last_df_yedek.csv',index= False)
# yeni df_i last_df_.csv seklinde atama 
df_concanated.to_csv('/Users/icy/Desktop/sofa/sofa_all_works/csvler/last_df_kontrol.csv', index= False)

/var/folders/ll/svcq3_v57v9dwq0lwqhp4vpc0000gn/T/ipykernel_52068/2463143209.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_week_matches['kg_var'].fillna(0, inplace= True)
/var/folders/ll/svcq3_v57v9dwq0lwqhp4vpc0000gn/T/ipykernel_52068/2463143209.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

In [110]:
df_concanated.isnull().sum().sum()

np.int64(0)

In [5]:
df = pd.read_csv('/Users/icy/sofascore/sofa_calisma/csvler/last_df.csv')

In [7]:
df.isnull().sum().sum()

0